In [1]:
!pip install plotly --root-user-action=ignore


In [2]:
!pip install pycaret[full] --root-user-action=ignore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.9/109.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 58.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 15.3 MB/s eta 0:00:00

In [3]:
!pip install lightgbm
!pip install --upgrade lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


# <p style="padding:10px;background-color:#A020F0;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Import all you need</p>

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
from pycaret.regression import *
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import joblib
import lightgbm as lgb


import warnings
warnings.filterwarnings('ignore')

# <p style="padding:10px;background-color:#A020F0;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Feature Selection</p>

In [5]:
df = pd.read_csv('/kaggle/input/salary-developers/survey_results_public.csv')

important_cols = [
    'Age', 'Gender',
    'EdLevel', 'YearsCodePro',
    'ConvertedCompYearly',
    'Country'
]

df = df[important_cols]
df.head()

,Age,Gender,EdLevel,YearsCodePro,ConvertedCompYearly,Country
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Canada
2,25-34 years old,Man,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,40205.0,United Kingdom of Great Britain and Northern I...
3,35-44 years old,Man,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",17,215232.0,Israel
4,25-34 years old,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",3,NaN,United States of America


# <p style="padding:10px;background-color:#A020F0;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">EDA</p>

In [6]:
df.describe()

,ConvertedCompYearly
count,3.807100e+04
mean,1.707613e+05
std,7.814132e+05
min,1.000000e+00
25%,3.583200e+04
50%,6.784500e+04
75%,1.200000e+05
max,5.000000e+07


In [7]:
def value_c(x):
    for i in x.columns:
        if x[i].dtypes == 'object':
            print(df[i].value_counts() , 3*'\n')
    print(df.shape)
value_c(df)

Age
25-34 years old       28112
18-24 years old       16646
35-44 years old       13988
45-54 years old        5281
Under 18 years old     3866
55-64 years old        1978
65 years or older       554
Prefer not to say       521
Name: count, dtype: int64 



Gender
Man                                                                                   64607
Woman                                                                                  3399
Prefer not to say                                                                      1172
Non-binary, genderqueer, or gender non-conforming                                       704
Or, in your own words:                                                                  279
Man;Non-binary, genderqueer, or gender non-conforming                                   235
Man;Or, in your own words:                                                              171
Woman;Non-binary, genderqueer, or gender non-conforming                                 160

# <p style="padding:10px;background-color:#A020F0;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Preprocessing </p>

In [8]:
df = df.dropna()

df=df[df['Gender'].isin(['Man','Woman'])]


In [9]:
df = df[~df['Age'].isin(['Prefer not to say'])]

def categorize_age(age):
    if age in ['25-34 years old', '18-24 years old']:
        return 'Young'
    elif age in ['35-44 years old', '45-54 years old']:
        return 'Adult'
    elif age == 'Under 18 years old':
        return 'Teenager'
    elif age in ['55-64 years old', '65 years or older']:
        return 'Elderly'


df['Age'] = df['Age'].apply(categorize_age)

In [10]:
df = df.rename({"ConvertedCompYearly": "Salary"}, axis=1)


In [11]:
def shorten_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            categorical_map[categories.index[i]] = 'Other'
    return categorical_map

country_map = shorten_categories(df.Country.value_counts(), 199)
df['Country'] = df['Country'].map(country_map)
df = df[df['Country'] != 'Other']

In [12]:
def categorize_edlevel(edlevel):
    if "Bachelor’s degree" in edlevel:
        return "Bachelor’s"
    elif "Master’s degree" in edlevel:
        return "Master’s"
    elif "doctoral degree" in edlevel:
        return "doctoral"
    elif "Primary/elementary school" in edlevel or "Secondary school" in edlevel:
        return "students / school"
    else:
        return "Other"

# Apply the categorization
df['EdLevel'] = df['EdLevel'].apply(categorize_edlevel)
df = df[df['EdLevel'] != 'Other']

In [13]:
def clean_experience(x):
    if x ==  'More than 50 years':
        return 50
    if x == 'Less than 1 year':
        return 0.5
    return float(x)

df['YearsCodePro'] = df['YearsCodePro'].apply(clean_experience)

# <p style="padding:10px;background-color:#A020F0;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Visualization</p>

In [14]:
import plotly.express as ex
ex.violin(x='Salary', data_frame=df)

In [15]:
Q1 = df.Salary.quantile(0.25)
Q3 = df.Salary.quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR
lower_limit, upper_limit

(-88357.5, 261014.5)

In [16]:
df.shape

(26243, 6)

In [17]:
df = df[(df.Salary>lower_limit)&(df.Salary<upper_limit)]
df.shape

(24253, 6)

In [18]:
def uniquer(x):
    for i in x.columns:
        if x[i].dtypes == 'object':
            print(df[i].unique() , 3*'\n')
    print(df.shape)
uniquer(df)

['Young' 'Adult' 'Elderly' 'Teenager'] 



['Man' 'Woman'] 



['Master’s' 'Bachelor’s' 'students / school' 'doctoral'] 



['United Kingdom of Great Britain and Northern Ireland' 'Israel'
 'Netherlands' 'United States of America' 'Czech Republic' 'Austria'
 'Italy' 'Germany' 'Poland' 'Norway' 'Canada' 'Brazil' 'Sweden' 'France'
 'Spain' 'Romania' 'India' 'Belgium' 'Russian Federation' 'Mexico'
 'Switzerland' 'South Africa' 'Finland' 'Denmark' 'Australia' 'Greece'
 'Portugal' 'Hungary' 'Bangladesh' 'Ukraine' 'Pakistan'
 'Iran, Islamic Republic of...' 'Turkey' 'Argentina' 'New Zealand'] 



(24253, 6)


In [19]:
df.head()

,Age,Gender,EdLevel,YearsCodePro,Salary,Country
2,Young,Man,Master’s,5.0,40205.0,United Kingdom of Great Britain and Northern I...
3,Adult,Man,Bachelor’s,17.0,215232.0,Israel
8,Young,Woman,Master’s,6.0,49056.0,Netherlands
10,Young,Man,Bachelor’s,2.0,60307.0,United Kingdom of Great Britain and Northern I...
11,Adult,Man,Bachelor’s,10.0,194400.0,United States of America


In [20]:
df.isnull().sum()

Age             0
Gender          0
EdLevel         0
YearsCodePro    0
Salary          0
Country         0
dtype: int64

# <p style="padding:10px;background-color:#A020F0;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Find best model by pycaret</p>

In [21]:
setup(data = df,target = 'Salary',session_id = 85)

,Description,Value
0,Session id,85
1,Target,Salary
2,Target type,Regression
3,Original data shape,"(24253, 6)"
4,Transformed data shape,"(24253, 12)"
5,Transformed train set shape,"(16977, 12)"
6,Transformed test set shape,"(7276, 12)"
7,Numeric features,1
8,Categorical features,4
9,Preprocess,True


In [22]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,25619.6635,1343880259.8045,36630.6364,0.5711,0.7261,12.3672,0.3110
lightgbm,Light Gradient Boosting Machine,25768.6661,1360496865.1219,36860.2240,0.5657,0.7261,11.4507,14.0060
catboost,CatBoost Regressor,25826.8866,1369148261.0525,36977.4689,0.5630,0.7265,9.5575,1.8430
br,Bayesian Ridge,26615.9351,1410889502.5577,37536.3793,0.5498,0.7379,14.7530,0.0740
ridge,Ridge Regression,26616.8446,1410962876.8835,37537.3965,0.5497,0.7383,14.7707,0.0720
lr,Linear Regression,26617.0096,1410976365.7666,37537.5783,0.5497,0.7384,14.7723,0.6490
lar,Least Angle Regression,26618.1811,1411085954.6573,37539.0313,0.5497,0.7384,14.7740,0.0790
llar,Lasso Least Angle Regression,26616.7228,1410972784.2532,37537.5242,0.5497,0.7381,14.7732,0.0710
lasso,Lasso Regression,26616.7233,1410972784.4078,37537.5241,0.5497,0.7381,14.7732,0.0750
xgboost,Extreme Gradient Boosting,26365.6816,1427727577.6000,37764.5664,0.5442,0.7352,11.4402,0.1300


GradientBoostingRegressor(random_state=85)

In [23]:
df.head()

,Age,Gender,EdLevel,YearsCodePro,Salary,Country
2,Young,Man,Master’s,5.0,40205.0,United Kingdom of Great Britain and Northern I...
3,Adult,Man,Bachelor’s,17.0,215232.0,Israel
8,Young,Woman,Master’s,6.0,49056.0,Netherlands
10,Young,Man,Bachelor’s,2.0,60307.0,United Kingdom of Great Britain and Northern I...
11,Adult,Man,Bachelor’s,10.0,194400.0,United States of America


In [24]:
uniquer(df)

['Young' 'Adult' 'Elderly' 'Teenager'] 



['Man' 'Woman'] 



['Master’s' 'Bachelor’s' 'students / school' 'doctoral'] 



['United Kingdom of Great Britain and Northern Ireland' 'Israel'
 'Netherlands' 'United States of America' 'Czech Republic' 'Austria'
 'Italy' 'Germany' 'Poland' 'Norway' 'Canada' 'Brazil' 'Sweden' 'France'
 'Spain' 'Romania' 'India' 'Belgium' 'Russian Federation' 'Mexico'
 'Switzerland' 'South Africa' 'Finland' 'Denmark' 'Australia' 'Greece'
 'Portugal' 'Hungary' 'Bangladesh' 'Ukraine' 'Pakistan'
 'Iran, Islamic Republic of...' 'Turkey' 'Argentina' 'New Zealand'] 



(24253, 6)


In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

df['Gender'].replace({'Man': 1, 'Woman': 0}, inplace=True)
df['EdLevel'] = df['EdLevel'].map({
    'Master’s': 3,
    'Bachelor’s': 2,
    'students / school': 1,
    'doctoral': 4
})
df['Age'] = df['Age'].map({
    'Young': 2,
    'Adult': 3,
    'Elderly': 4,
    'Teenager': 1
})
df = pd.get_dummies(df, columns=['Country'],dtype=int)
df.head()

,Age,Gender,EdLevel,YearsCodePro,Salary,Country_Argentina,Country_Australia,Country_Austria,Country_Bangladesh,Country_Belgium,...,Country_Romania,Country_Russian Federation,Country_South Africa,Country_Spain,Country_Sweden,Country_Switzerland,Country_Turkey,Country_Ukraine,Country_United Kingdom of Great Britain and Northern Ireland,Country_United States of America
2,2,1,3,5.0,40205.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,1,2,17.0,215232.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2,0,3,6.0,49056.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,2,1,2,2.0,60307.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11,3,1,2,10.0,194400.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [26]:
df.columns= df.columns.str.replace('Country_','')
df.columns= df.columns.str.replace('Iran, Islamic Republic of...','')

In [27]:
x=df.drop(columns='Salary' , axis=1)
y=df['Salary']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.10, random_state=85)

# <p style="padding:10px;background-color:#A020F0;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">build best model</p>

In [29]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def lgb_tuning(X_train, y_train, n_iter=20):
    import lightgbm as lgb
    from sklearn.model_selection import RandomizedSearchCV

    # Reset feature names to prevent mismatches
    X_train = X_train.copy()
    X_train.columns = [f'feature_{i}' for i in range(X_train.shape[1])]
    
    param_distributions = {
        'n_estimators': [100, 200, 500, 1000],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7, 9, -1],
        'num_leaves': [31, 63, 127, 255],
        'min_child_samples': [5, 10, 20, 50],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'reg_alpha': [0, 0.1, 0.5, 1.0],
        'reg_lambda': [0, 0.1, 0.5, 1.0]
    }

    random_search = RandomizedSearchCV(
        lgb.LGBMRegressor(random_state=42, feature_name='auto'),
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=5,
        scoring='r2',
        n_jobs=-1,
        verbose=1,
        random_state=42
    )
    
    random_search.fit(X_train, y_train)

    # Get the best model and parameters
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    best_score = random_search.best_score_

    # Make predictions
    y_pred = best_model.predict(X_test)

    # Evaluate performance
    test_r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)

    eval_metrics = {
        'Test R² Score': test_r2,
        'RMSE': rmse,
        'MAE': mae
    }

    print(best_score)
    print(eval_metrics)

    
    print("\n📊 **Model Evaluation Metrics:**")
    for metric, value in eval_metrics.items():
        print(f"{metric}: {value:.4f}")

    return best_model


In [30]:
bmodel = lgb_tuning(X_train,y_train,n_iter=15)


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[LightGBM] [Warning] Unknown parameter: feature_name
[LightGBM] [Warning] Unknown parameter: feature_name
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000238 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 132
[LightGBM] [Info] Number of data points in the train set: 21827, number of used features: 39
[LightGBM] [Info] Start training from score 81590.985431
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

# <p style="padding:10px;background-color:#A020F0;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">save model</p>

In [31]:
joblib.dump(bmodel,'pre_Salary.joblib')

['pre_Salary.joblib']